In [1]:
import re
import pandas as pd
import pandas_gbq

In [2]:
project_id='testtaskproject-265509'
table_id = 'your_dataset' + '.orders_products_merged'

## Load data to df.

In [3]:
products = pd.read_csv('data_source/products.csv')

In [4]:
orders = pd.read_csv('data_source/orders_s.csv')

## Clean up products data.

### Logic:
    - product data looks ok, except of product_id column, so need to proccess it.

In [5]:
products.product_id = pd.to_numeric(products.product_id, downcast='integer')

## Clean up orders data.

### Logic:
   1. Cast order_created_datetime field to datetime
   2. Cast sum field to numeric type(float)
   3. Create 'wrong_name' bool type column for marking bad formatted/duplicated names.
   4. Clean up and cast product_id.
   5. Remove duplicated data.
   6. Merge orders and product tables by product_id.
   7. Write merged data to csv for future downloading to Google Cloud.

In [6]:
# Cast order_created_datetime to datetime.

orders.order_created_datetime = pd.to_datetime(orders.order_created_datetime)

In [7]:
# Cast sum field to numeric type(float)

orders['sum'] = orders['sum'].apply(lambda x: float(x.replace(',', '.')))

In [8]:
# This function return true(mean: name format is wrong) 
# if (name, surname, patronymic) duplicated or length of surname/patronymic too short. 

def check_duplicates(row):
    name = row['name'].strip()
    surname = row['surname'].strip()
    patronymic = row['patronymic'].strip()
    if (name == surname or surname == patronymic or name == patronymic):
        return True
    elif (len(surname) < 2 or len(patronymic) < 2):
        return True
    else:
        return False

In [9]:
# Create 'wrong_name' bool type column for marking bad formatted/duplicated names.

orders['wrong_name'] = orders[['name', 'surname', 'patronymic']].apply(axis=1, func=check_duplicates)

In [10]:
# This function cut number(id) from product_id string(object) and casts it to int.

def clean_product_id(product_id):
    if product_id.isnumeric():
        return int(product_id)
    else:
        product_id = re.findall(r'\d+', product_id)[0]
        return int(product_id)

In [11]:
# Clean up and cast product_id.

orders.product_id = orders.product_id.apply(func=clean_product_id)

In [12]:
# Remove duplicated data

subset = list(orders.columns)
subset.remove('Unnamed: 0')
orders.drop_duplicates(subset=subset,inplace=True)


In [13]:
# merge orders and product tables by product_id.

data = pd.merge(orders, products, on='product_id')

In [14]:
# Need to rename Unnamed: 0 column, because gbq doesn't works with spaces in table column names.

data.rename(columns={'Unnamed: 0': 'unnamed'}, inplace=True)

In [15]:
# Write merged data to csv.

data.to_csv('orders_products_cleaned_merged.csv', index=False)

### Upload  merged and cleaned up data to gbq

In [16]:
pandas_gbq.to_gbq(data, table_id, project_id=project_id, if_exists='replace')